In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/mnt/d/projects/large_projects/CricketSemantics/cricket_data_scrapper/cricket_data.csv")

In [3]:
df

,match,ball,commentry
0,ENGLAND vs NEW ZEALAND,36.2,"36.2 Moeen Ali to Ravindra, 1 run, clipped awa..."
1,ENGLAND vs NEW ZEALAND,36.1,"36.1 Moeen Ali to Conway, 1 run, shortish deli..."
2,ENGLAND vs NEW ZEALAND,35.6,"35.6 Livingstone to Ravindra, FOUR, short and ..."
3,BANGLADESH vs AFGHANISTAN,34.4,"34.4Naveen-ul-Haq to Shanto, FOUR, Shanto hits..."
4,ENGLAND vs NEW ZEALAND,35.5,"35.5 Livingstone to Conway, 1 run, misfield at..."
...,...,...,...
26116,AUSTRALIA vs NEW ZEALAND,0.5,"0.5 Matt Henry to Warner, FOUR, now Warner hop..."
26117,AUSTRALIA vs NEW ZEALAND,0.4,"0.4 Matt Henry to Warner, no run, short of len..."
26118,AUSTRALIA vs NEW ZEALAND,0.3,"0.3 Matt Henry to Warner, FOUR, width and cut ..."
26119,AUSTRALIA vs NEW ZEALAND,0.2,"0.2 Matt Henry to Warner, no run, bit of bounc..."


In [4]:
texts = df['commentry'].tolist()

In [5]:
len(texts)

26121

In [6]:
from sentence_transformers import SentenceTransformer

text = df['commentry']
encoder = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
vector = encoder.encode(text)

/home/arjunrao/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
print(vector)

[[-0.3069382  -0.49472904 -0.28060228 ...  0.03433657 -0.04910103
  -0.22666326]
 [-0.34834692 -0.8107878  -0.24893562 ...  0.12750685 -0.14558958
  -0.17948112]
 [-0.11351443 -0.6346422  -0.24114348 ...  0.13689336 -0.27959332
  -0.2406624 ]
 ...
 [ 0.11461562 -0.80141824 -0.17872243 ... -0.01341299  0.38732326
  -0.10957686]
 [-0.09158109 -0.51889133 -0.17674406 ...  0.0463693   0.21876432
  -0.0465444 ]
 [-0.15903863 -0.72816944 -0.27062908 ...  0.03926188 -0.02273992
  -0.1073662 ]]


In [8]:
len(vector)

26121

In [9]:
vector.shape[1]

768

In [10]:
import faiss
from faiss import write_index, read_index

vector_dimention = vector.shape[1]
index = faiss.IndexFlatL2(vector_dimention)
faiss.normalize_L2(vector)
index.add(vector)

write_index(index,"cricketSemanticSearch.index")



In [11]:
import numpy as np
import pandas as pd
from icecream import ic
import faiss
from faiss import read_index
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
index = read_index("cricketSemanticSearch.index")
df = pd.read_csv("/mnt/d/projects/large_projects/CricketSemantics/cricket_data_scrapper/cricket_data.csv")

def search(text):
    search_text = text
    search_vector = encoder.encode(search_text)
    _vector = np.array([search_vector])
    faiss.normalize_L2(_vector)
    k = index.ntotal
    ic(k)
    distances,ann = index.search(_vector,k=k)
    ic(distances)
    ic(ann[0][0:5])

    for x in ann[0][0:5]:
        ic(df.iloc[x]['commentry'])

    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [12]:
search("afghanistan win this game")

ic| k: 26121
ic| distances: array([[0.67458737, 0.7261881 , 0.7711846 , ..., 1.6805038 , 1.6860095 ,
                       1.6921207 ]], dtype=float32)
ic| ann[0][0:5]: array([  206, 25072, 15866,   292,    64])
ic| df.iloc[x]['commentry']: ("45.2 Chameera to Azmatullah, 1 run, that's it. Asalanka drops a sitter at "
                              'deep square and that will do for Afghanistan. It was slogged high into the '
                              'night sky and went straight towards the fielder in the deep, Asalanka ran '
                              "forward but couldn't gather it cleanly. Afghanistan win by 7 wickets and "
                              'move to 5th slot. This is their 3rd win in 4 games')
ic| df.iloc[x]['commentry']: ("7.6 Hazlewood to Gurbaz, out Caught by Starc!! Oh that's really loose from "
                              "Gurbaz. He's thrown it away he's the first one to be gutted about it. This "
                              'was a harmless short of leng

match                              NEW ZEALAND vs SOUTH AFRICA
ball                                                      34.3
commentry    34.3 Maharaj to Glenn Phillips, FOUR, fifty fo...
Name: 300, dtype: object